### MLlib package of PySpark  
스파크 2.0부터 ML 패키지는 데이터프레임에 대해 작동
MLlist 패키지는 RDD에 대해서 작동 함.  

MLlib은 전체적으로 크게 세단계의 머신러닝 기능으로 구분 된다.  
- 데이터 전처리 : 피처 추출, 변형, 선택, 카테고리 피처에 대한 해석, 자연어 처리  
- 머신러닝 알고리즘 : 몇몇 유명, 고급 레벨인 회귀, 분류, 군집 알고리즘 지원  
- 유틸리티 : 기술통계, 카이스퀘어(ChiSquare)테스트, 선행대수, 모델평가  



In [11]:
import findspark, pyspark
findspark.find()


In [16]:
import os
os.getcwd()

'c:\\Users\\admin\\Downloads'

In [2]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession 
conf = pyspark.SparkConf() \
                .setAppName('appName') \
                .setMaster('local[2]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
# spark.stop()

US 2014 ~ 2015 출생 데이터의 일부  
원본 데이터는 300개의 퓨처로 구성, 그 중 85개 선별  
799만개의 데이터 중 45429개 균등 샘플링  
INFANT_ALIVE_AT_REPORT 가 1인지 0인지 예측 하는 것이 목표.  

In [3]:
import pyspark.sql.types as typ
labels = [
    ('INFANT_ALIVE_AT_REPORT', typ.StringType()),
    ('BIRTH_YEAR', typ.IntegerType()),
    ('BIRTH_MONTH', typ.IntegerType()),
    ('BIRTH_PLACE', typ.StringType()),
    ('MOTHER_AGE_YEARS', typ.IntegerType()),
    ('MOTHER_RACE_6CODE', typ.StringType()),
    ('MOTHER_EDUCATION', typ.StringType()),
    ('FATHER_COMBINED_AGE', typ.IntegerType()),
    ('FATHER_EDUCATION', typ.StringType()),
    ('MONTH_PRECARE_RECODE', typ.StringType()),
    ('CIG_BEFORE', typ.IntegerType()),
    ('CIG_1_TRI', typ.IntegerType()),
    ('CIG_2_TRI', typ.IntegerType()),
    ('CIG_3_TRI', typ.IntegerType()),
    ('MOTHER_HEIGHT_IN', typ.IntegerType()),
    ('MOTHER_BMI_RECODE', typ.IntegerType()),
    ('MOTHER_PRE_WEIGHT', typ.IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', typ.IntegerType()),
    ('MOTHER_WEIGHT_GAIN', typ.IntegerType()),
    ('DIABETES_PRE', typ.StringType()),
    ('DIABETES_GEST', typ.StringType()),
    ('HYP_TENS_PRE', typ.StringType()),
    ('HYP_TENS_GEST', typ.StringType()),
    ('PREV_BIRTH_PRETERM', typ.StringType()),
    ('NO_RISK', typ.StringType()),
    ('NO_INFECTIONS_REPORTED', typ.StringType()),
    ('LABOR_IND', typ.StringType()),
    ('LABOR_AUGM', typ.StringType()),
    ('STEROIDS', typ.StringType()),
    ('ANTIBIOTICS', typ.StringType()),
    ('ANESTHESIA', typ.StringType()),
    ('DELIV_METHOD_RECODE_COMB', typ.StringType()),
    ('ATTENDANT_BIRTH', typ.StringType()),
    ('APGAR_5', typ.IntegerType()),
    ('APGAR_5_RECODE', typ.StringType()),
    ('APGAR_10', typ.IntegerType()),
    ('APGAR_10_RECODE', typ.StringType()),
    ('INFANT_SEX', typ.StringType()),
    ('OBSTETRIC_GESTATION_WEEKS', typ.IntegerType()),
    ('INFANT_WEIGHT_GRAMS', typ.IntegerType()),
    ('INFANT_ASSIST_VENTI', typ.StringType()),
    ('INFANT_ASSIST_VENTI_6HRS', typ.StringType()),
    ('INFANT_NICU_ADMISSION', typ.StringType()),
    ('INFANT_SURFACANT', typ.StringType()),
    ('INFANT_ANTIBIOTICS', typ.StringType()),
    ('INFANT_SEIZURES', typ.StringType()),
    ('INFANT_NO_ABNORMALITIES', typ.StringType()),
    ('INFANT_ANCEPHALY', typ.StringType()),
    ('INFANT_MENINGOMYELOCELE', typ.StringType()),
    ('INFANT_LIMB_REDUCTION', typ.StringType()),
    ('INFANT_DOWN_SYNDROME', typ.StringType()),
    ('INFANT_SUSPECTED_CHROMOSOMAL_DISORDER', typ.StringType()),
    ('INFANT_NO_CONGENITAL_ANOMALIES_CHECKED', typ.StringType()),
    ('INFANT_BREASTFED', typ.StringType())
]

schema = typ.StructType([typ.StructField(e[0], e[1], False) for e in labels])

In [14]:
births = spark.read.csv(base_path+'births_train.csv', header=True, schema=schema)

AnalysisException: 'Path does not exist: file:/c:/Users/data/RDD_example/births_train.csv;'

In [ ]:
type(births)

In [ ]:
births.printSchema()

In [ ]:
selected_features = [
    'INFANT_ALIVE_AT_REPORT', 
    'BIRTH_PLACE', 
    'MOTHER_AGE_YEARS', 
    'FATHER_COMBINED_AGE', 
    'CIG_BEFORE', 
    'CIG_1_TRI', 
    'CIG_2_TRI', 
    'CIG_3_TRI', 
    'MOTHER_HEIGHT_IN', 
    'MOTHER_PRE_WEIGHT', 
    'MOTHER_DELIVERY_WEIGHT', 
    'MOTHER_WEIGHT_GAIN', 
    'DIABETES_PRE', 
    'DIABETES_GEST', 
    'HYP_TENS_PRE', 
    'HYP_TENS_GEST', 
    'PREV_BIRTH_PRETERM'
]

births_trimmed = births.select(selected_features)

In [ ]:
births_trimmed.head()

데이터셋에는 Yes/No/Unkown값을 가진 피처들이 매우 많음.  
Yes는 1, 나머지는 0으로 변환  

어머니의 흡연량 관련 레코드 :  
0 : 임신 기간 동안 금연  
1-97 : 한개 ~ 97개 사이  
98 : 98개 이상  
99 : 알 수 없음  

In [ ]:
# Y : yes
# N : no
# U : unkown
recode_dictionary = {
    'YNU': {
        'Y': 1,
        'N': 0,
        'U': 0
    }
}

In [ ]:
import pyspark.sql.functions as func

def recode(col, key):        
    return recode_dictionary[key][col] 

def correct_cig(feat):
    return func \
        .when(func.col(feat) != 99, func.col(feat))\
        .otherwise(0)

rec_integer = func.udf(recode, typ.IntegerType())

In [9]:
births_transformed = births_trimmed \
    .withColumn('CIG_BEFORE', correct_cig('CIG_BEFORE'))\
    .withColumn('CIG_1_TRI', correct_cig('CIG_1_TRI'))\
    .withColumn('CIG_2_TRI', correct_cig('CIG_2_TRI'))\
    .withColumn('CIG_3_TRI', correct_cig('CIG_3_TRI'))

In [ ]:
cols = [(col.name, col.dataType) for col in births_trimmed.schema]
cols

In [ ]:
births_trimmed.schema

In [12]:
YNU_cols = []

for i, s in enumerate(cols):
    if s[1] == typ.StringType():
        dis = births.select(s[0]) \
            .distinct() \
            .rdd \
            .map(lambda row: row[0]) \
            .collect()

        if 'Y' in dis:
            YNU_cols.append(s[0])

In [13]:
births.select('INFANT_ALIVE_AT_REPORT') \
            .distinct() \
            .rdd \
            .map(lambda row: row[0]) \
            .collect()

['Y', 'N']

In [14]:
births.select('BIRTH_PLACE') \
            .distinct() \
            .rdd \
            .map(lambda row: row[0]) \
            .collect()

['7', '3', '5', '6', '9', '1', '4', '2']

In [20]:
births.select([
    'INFANT_NICU_ADMISSION', 
    rec_integer(
        'INFANT_NICU_ADMISSION', 
        func.lit('YNU')).alias('INFANT_NICU_ADMISSION_RECODE')
    ]).take(5)

[Row(INFANT_NICU_ADMISSION='Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION='Y', INFANT_NICU_ADMISSION_RECODE=1),
 Row(INFANT_NICU_ADMISSION='U', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION='N', INFANT_NICU_ADMISSION_RECODE=0),
 Row(INFANT_NICU_ADMISSION='U', INFANT_NICU_ADMISSION_RECODE=0)]

In [21]:
exprs_YNU = [
    rec_integer(x, func.lit('YNU')).alias(x)
    if x in YNU_cols
    else x
    for x in births_transformed.columns
]

In [ ]:
exprs_YNU

In [23]:
births_transformed = births_transformed.select(exprs_YNU)

In [24]:
births_transformed.select(YNU_cols[-5:]).show(5)

+------------+-------------+------------+-------------+------------------+
|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+------------+-------------+------------+-------------+------------------+
|           0|            0|           0|            0|                 0|
|           0|            0|           0|            0|                 0|
|           0|            0|           0|            0|                 0|
|           0|            0|           0|            0|                 1|
|           0|            0|           0|            0|                 0|
+------------+-------------+------------+-------------+------------------+
only showing top 5 rows



In [25]:
YNU_cols[-5:]

['DIABETES_PRE',
 'DIABETES_GEST',
 'HYP_TENS_PRE',
 'HYP_TENS_GEST',
 'PREV_BIRTH_PRETERM']

In [27]:
births_transformed.columns

['INFANT_ALIVE_AT_REPORT',
 'BIRTH_PLACE',
 'MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_BEFORE',
 'CIG_1_TRI',
 'CIG_2_TRI',
 'CIG_3_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'MOTHER_DELIVERY_WEIGHT',
 'MOTHER_WEIGHT_GAIN',
 'DIABETES_PRE',
 'DIABETES_GEST',
 'HYP_TENS_PRE',
 'HYP_TENS_GEST',
 'PREV_BIRTH_PRETERM']

In [26]:
# Descriptive statistids

import pyspark.mllib.stat as st
import numpy as np

In [29]:
numeric_cols = [
 'MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_BEFORE',
 'CIG_1_TRI',
 'CIG_2_TRI',
 'CIG_3_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'MOTHER_DELIVERY_WEIGHT',
 'MOTHER_WEIGHT_GAIN'
]

In [30]:
numeric_rdd = births_transformed.select(numeric_cols).rdd.map(lambda row: [e for e in row])

In [31]:
mllib_stats = st.Statistics.colStats(numeric_rdd)

In [32]:
for col, m, v in zip(numeric_cols, mllib_stats.mean(), mllib_stats.variance()):
    print('{0}: \t{1:.2f} \t{2:.2f}'.format(col, m, np.sqrt(v)))

MOTHER_AGE_YEARS: 	28.30 	6.08
FATHER_COMBINED_AGE: 	44.55 	27.55
CIG_BEFORE: 	1.43 	5.18
CIG_1_TRI: 	0.91 	3.83
CIG_2_TRI: 	0.70 	3.31
CIG_3_TRI: 	0.58 	3.11
MOTHER_HEIGHT_IN: 	65.12 	6.45
MOTHER_PRE_WEIGHT: 	214.50 	210.21
MOTHER_DELIVERY_WEIGHT: 	223.63 	180.01
MOTHER_WEIGHT_GAIN: 	30.74 	26.23


colStats() 한수는 기술통계를 샘플링 해서 계산한다.  
실제 데이터에서는 큰 문제는 없다. 데이터셋이 100개 미만일 경우에는 문제가 될 수 있다.  

collStats()함수는 RDD 데이터를 취해 기술 통계를 계산한다.  
MulhvariateStahshcalSummary 객체를 리턴

- count() : 데이터 행 갯수  
- max() : 최대값   
- mean() : 평균  
- min() : 최솟값  
- numNonzeros() : 0이 아닌 값의 갯수  
- variance() : 분산  
- norml1() : L1-Norm 값  
- norml2() : L2-Norm 값  

L2 Loss는 직관적으로 오차의 제곱을 더함, Outlier에 큰 영향을 받는다.  
L1 Loss가 L2 Loss에 비해 Outlier에 대하여 더 Robust 하다.  
Outlier가 적당히 무시되길 원한다면 L1 Loss를, Outlier에 신경을 써야 한다면 L2 Loss를 사용.  



In [34]:
categorical_cols = [e for e in births_transformed.columns if e not in numeric_cols]

In [35]:
categorical_rdd = births_transformed.select(categorical_cols)\
                    .rdd.map(lambda row:[e for e in row])

In [36]:
for i, col in enumerate(categorical_cols):
    agg = categorical_rdd.groupBy(lambda row: row[i])\
                        .map(lambda row: (row[0], len(row[1])))
    print(col, sorted(agg.collect(), key=lambda el: el[1], reverse=True))

INFANT_ALIVE_AT_REPORT [(1, 23349), (0, 22080)]
BIRTH_PLACE [('1', 44558), ('4', 327), ('3', 224), ('2', 136), ('7', 91), ('5', 74), ('6', 11), ('9', 8)]
DIABETES_PRE [(0, 44881), (1, 548)]
DIABETES_GEST [(0, 43451), (1, 1978)]
HYP_TENS_PRE [(0, 44348), (1, 1081)]
HYP_TENS_GEST [(0, 43302), (1, 2127)]
PREV_BIRTH_PRETERM [(0, 43088), (1, 2341)]


In [37]:
corrs = st.Statistics.corr(numeric_rdd)

In [38]:
for i, el in enumerate(corrs > 0.5):
    correlated = [
        (numeric_cols[j], corrs[i][j])
        for j, e in enumerate(el)
        if e == 1.0 and j != i
    ]
    if len(correlated) > 0:
        for e in correlated:
            print('{0}-to-{1}: {2:.2f}' \
                 .format(numeric_cols[i], e[0], e[1]))

CIG_BEFORE-to-CIG_1_TRI: 0.83
CIG_BEFORE-to-CIG_2_TRI: 0.72
CIG_BEFORE-to-CIG_3_TRI: 0.62
CIG_1_TRI-to-CIG_BEFORE: 0.83
CIG_1_TRI-to-CIG_2_TRI: 0.87
CIG_1_TRI-to-CIG_3_TRI: 0.76
CIG_2_TRI-to-CIG_BEFORE: 0.72
CIG_2_TRI-to-CIG_1_TRI: 0.87
CIG_2_TRI-to-CIG_3_TRI: 0.89
CIG_3_TRI-to-CIG_BEFORE: 0.62
CIG_3_TRI-to-CIG_1_TRI: 0.76
CIG_3_TRI-to-CIG_2_TRI: 0.89
MOTHER_PRE_WEIGHT-to-MOTHER_DELIVERY_WEIGHT: 0.54
MOTHER_PRE_WEIGHT-to-MOTHER_WEIGHT_GAIN: 0.65
MOTHER_DELIVERY_WEIGHT-to-MOTHER_PRE_WEIGHT: 0.54
MOTHER_DELIVERY_WEIGHT-to-MOTHER_WEIGHT_GAIN: 0.60
MOTHER_WEIGHT_GAIN-to-MOTHER_PRE_WEIGHT: 0.65
MOTHER_WEIGHT_GAIN-to-MOTHER_DELIVERY_WEIGHT: 0.60


In [39]:
births_transformed.columns

['INFANT_ALIVE_AT_REPORT',
 'BIRTH_PLACE',
 'MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_BEFORE',
 'CIG_1_TRI',
 'CIG_2_TRI',
 'CIG_3_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'MOTHER_DELIVERY_WEIGHT',
 'MOTHER_WEIGHT_GAIN',
 'DIABETES_PRE',
 'DIABETES_GEST',
 'HYP_TENS_PRE',
 'HYP_TENS_GEST',
 'PREV_BIRTH_PRETERM']

In [40]:
features_to_keep = [
 'INFANT_ALIVE_AT_REPORT',
 'BIRTH_PLACE',
 'MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_1_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'DIABETES_PRE',
 'DIABETES_GEST',
 'HYP_TENS_PRE',
 'HYP_TENS_GEST',
 'PREV_BIRTH_PRETERM'
]

In [41]:
births_transformed = births_transformed.select([e for e in features_to_keep])

In [42]:
# 카데고리 피처에 대한 확인
import pyspark.mllib.linalg as ln

In [44]:
for cat in categorical_cols[1:]:
    agg = births_transformed.groupby('INFANT_ALIVE_AT_REPORT') \
                            .pivot(cat) \
                            .count()
    agg_rdd = agg.rdd.map(lambda row: (row[1:])) \
                .flatMap(lambda row: [0 if e == None else e for e in row]) \
                .collect()
    row_length = len(agg.collect()[0]) - 1
    agg = ln.Matrices.dense(row_length, 2, agg_rdd)
    
    test = st.Statistics.chiSqTest(agg)
    print(cat, round(test.pValue, 4))

BIRTH_PLACE 0.0
DIABETES_PRE 0.0
DIABETES_GEST 0.0
HYP_TENS_PRE 0.0
HYP_TENS_GEST 0.0
PREV_BIRTH_PRETERM 0.0


Matrices.dense() 함수
첫번째 파라미터 : 행렬에서 행의 수  
두번째 파라미터 : 카테고리 고유 값의 수  
세번째 파라미터 : 행렬로 변환 할 값들에 대한 리스트  

In [47]:
# final dataset
import pyspark.mllib.feature as ft
import pyspark.mllib.regression as reg

hashing = ft.HashingTF(7)
births_hashed = births_transformed.rdd\
                    .map(lambda row: [
                        list(hashing.transform(row[1]).toArray())
                            if col == 'BIRTH_PLACE'
                            else row[i]
                        for i, col in enumerate(features_to_keep)
                        ])\
                    .map(lambda row: [[e] if type(e) == int else e for e in row])\
                    .map(lambda row: [item for sublist in row for item in sublist])\
                    .map(lambda row: reg.LabeledPoint(row[0], ln.Vectors.dense(row[1:])))

In [48]:
# training and testing data
births_train, births_test = births_hashed.randomSplit([0.6, 0.4])

In [49]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
LR_Model = LogisticRegressionWithLBFGS.train(births_train, iterations=10)

In [51]:
LR_results = (
    births_test.map(lambda row: row.label)\
                .zip(LR_Model.predict(births_test.map(lambda row: row.features)))
).map(lambda row: (row[0], row[1] * 1.0))

In [ ]:
LR_results.collect()

In [54]:
# 모델 평가
import pyspark.mllib.evaluation as ev
LR_evaluation = ev.BinaryClassificationMetrics(LR_results)

print('Area under PR: {0:.2f}'.format(LR_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}'.format(LR_evaluation.areaUnderROC))
LR_evaluation.unpersist()

Area under PR: 0.80
Area under ROC: 0.63


In [55]:
# ChiSqSelector() 함수를 이용한 피처 선태
selector = ft.ChiSqSelector(4).fit(births_train)

In [56]:
topFeatures_train = (
    births_train.map(lambda row: row.label)\
            .zip(selector.transform(births_train.map(lambda row: row.features)))
).map(lambda row: reg.LabeledPoint(row[0], row[1]))

In [57]:
topFeatures_test = (
    births_test.map(lambda row: row.label)\
            .zip(selector.transform(births_test.map(lambda row: row.features)))
).map(lambda row: reg.LabeledPoint(row[0], row[1]))

In [58]:
# Random Forest
from pyspark.mllib.tree import RandomForest
RF_model = RandomForest.trainClassifier(data=topFeatures_train,
                                       numClasses=2,
                                       categoricalFeaturesInfo={},
                                       numTrees=6,
                                       featureSubsetStrategy='all',
                                       seed=5)

trainClassifier  
첫번째 : 트레인데이타  
두번째 : 타깃이 몇개인지  
세번째 : 딕셔너리타입 (키 : 학습데이터셋RDD의 카테고리 피처의 인덱스, 값: 각 카테고리 피처가 가지고있는 레벨 수  
네번째 : 랜덤 포레스트 모델에서 생성할 트리 수  

In [59]:
# 모델 성능 평가
RF_results = (
    topFeatures_test.map(lambda row: row.label)\
                .zip(RF_model.predict(topFeatures_test.map(lambda row: row.features)))
)

In [60]:
RF_evaluation = ev.BinaryClassificationMetrics(RF_results)

print('Area under PR: {0:.2f}'.format(RF_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}'.format(RF_evaluation.areaUnderROC))
RF_evaluation.unpersist()

Area under PR: 0.85
Area under ROC: 0.62
